In [29]:
from utils import make_need_tree,visualize_tree,make_storage,make_task_list,calc_start_end_time
import plotly.express as px
import pandas as pd
import json
import random
import copy

In [30]:
storage = make_storage()
storage['apple']=10
tasks=make_task_list(['hamburger','bacon and eggs','cheese','baked potato','pancake','pancake'
                      ],storage)
len(tasks)

80

In [31]:
# number of machines
sources={
    'Bakery':1, 'Chicken':2, 'Feed Mill':1, 'Field':8   , 'Raspberry bush':1,'Sugar Mill':1,'Cake Oven':1,
    'Cherry tree':2,'Cow':4,'BBQ Grill':1,'Dairy':1,"Pig":2,'Popcorn Pot':1,'Loom':1,'Sewing Machine':1,
    'Pie Oven':1
}
machines ={}
# make machines is: key is same as sources, value is a list of machine_id
# machine_id must be unique from other machines
count  = 0
for key,value in sources.items():
    machines[key]=list(range(count,count+value))
    count+=value
machines

{'Bakery': [0],
 'Chicken': [1, 2],
 'Feed Mill': [3],
 'Field': [4, 5, 6, 7, 8, 9, 10, 11],
 'Raspberry bush': [12],
 'Sugar Mill': [13],
 'Cake Oven': [14],
 'Cherry tree': [15, 16],
 'Cow': [17, 18, 19, 20],
 'BBQ Grill': [21],
 'Dairy': [22],
 'Pig': [23, 24],
 'Popcorn Pot': [25],
 'Loom': [26],
 'Sewing Machine': [27],
 'Pie Oven': [28]}

In [32]:
import random

def make_chromosome(tasks,machines):
    tasks = copy.deepcopy(tasks)
    random.shuffle(tasks)
    tasks.sort(key=lambda x:x['order'],reverse=True)
    for task in tasks:
        machine_id=random.choice(machines[task['source']])
        task['machine_id']=machine_id
    return tasks

def calc_fitness(chromosome):
    """calculate the fitness of a chromosome
    chromosome: a chromosome
    
    return: a fitness value
    """
    score = 0
    chromosome = calc_start_end_time(chromosome,machines)
    # maximum end_time
    max_end_time = max([task['end_time'] for task in chromosome])
    score += 1000000/max_end_time
    # for task in chromosome:
        # if task['target']:
            # score+=1000/task['end_time']
    return score
    
def mutation(chromosome):
    """mutate a chromosome
    chromosome: a chromosome
    
    return: a chromosome
    """
    chromosome = copy.deepcopy(chromosome)
    if random.random()<0.5:
        gen = random.choice(chromosome)
        new_machine_id = random.choice(machines[gen['source']])
        gen['machine_id']=new_machine_id
        
        gen1,gen2 = random.sample(chromosome,2)
        idx1 = chromosome.index(gen1)
        idx2 = chromosome.index(gen2)
        if idx1>idx2:
            gen1,gen2=gen2,gen1
        if gen1['order']>=gen2['order']:
            chromosome[idx1],chromosome[idx2]=chromosome[idx2],chromosome[idx1]
    return chromosome


In [33]:
# simple GA algorithm
pop_size = 50
iteration = 200
# generate inital population
population = []
for i in range(pop_size):
    population.append(make_chromosome(tasks,machines))

# calculate fitness
fitness = []
for chromosome in population:
    fitness.append(calc_fitness(chromosome))
    
for i in range(iteration):
    new_population = []
    # keep the best chromosome
    best_chromosome = population[fitness.index(max(fitness))]
    new_population.append(best_chromosome)
    for j in range(pop_size-1):
        # no cross over
        # mutation
        chromosome = mutation(best_chromosome)
        new_population.append(chromosome)
    population = new_population
    fitness = []
    for chromosome in population:
        fitness.append(calc_fitness(chromosome))
    best_chromosome = population[fitness.index(max(fitness))]
    print(f'iteration {i}, best fitness {max(fitness)}, best time {max([task["end_time"] for task in best_chromosome])}')

iteration 0, best fitness 25.44529262086514, best time 39300
iteration 1, best fitness 26.164311878597594, best time 38220
iteration 2, best fitness 26.58160552897395, best time 37620
iteration 3, best fitness 26.925148088314486, best time 37140
iteration 4, best fitness 27.233115468409586, best time 36720
iteration 5, best fitness 27.233115468409586, best time 36720
iteration 6, best fitness 27.233115468409586, best time 36720
iteration 7, best fitness 27.233115468409586, best time 36720
iteration 8, best fitness 27.233115468409586, best time 36720
iteration 9, best fitness 27.233115468409586, best time 36720
iteration 10, best fitness 27.233115468409586, best time 36720
iteration 11, best fitness 27.233115468409586, best time 36720
iteration 12, best fitness 27.917364600781685, best time 35820
iteration 13, best fitness 28.011204481792717, best time 35700
iteration 14, best fitness 28.24858757062147, best time 35400
iteration 15, best fitness 28.24858757062147, best time 35400
iterat

In [ ]:
# chromosome = make_chromosome(tasks,machines)
chromosome = population[fitness.index(max(fitness))]
test_tasks = calc_start_end_time(chromosome,machines)
df = pd.DataFrame(test_tasks)
# task is source + machine_id
df['task']=df['source']+' '+df['machine_id'].astype(str)
df['start'] = pd.to_datetime('today')+pd.to_timedelta(df['start_time'],unit='s')
df['end'] = pd.to_datetime('today')+pd.to_timedelta(df['end_time'],unit='s')
df.head()

,id,name,dependencies,duration,source,machine_id,target,order,start_time,end_time,task,start,end
0,53,soybean,[],1200,Field,7,False,5,0,1200,Field 7,2024-01-04 05:22:47.147383,2024-01-04 05:42:47.149239
1,52,corn,[],300,Field,6,False,5,0,300,Field 6,2024-01-04 05:22:47.147383,2024-01-04 05:27:47.149239
2,43,corn,[],300,Field,6,False,5,300,600,Field 6,2024-01-04 05:27:47.147383,2024-01-04 05:32:47.149239
3,54,soybean,[],1200,Field,10,False,5,0,1200,Field 10,2024-01-04 05:22:47.147383,2024-01-04 05:42:47.149239
4,77,egg,"[73, 74, 75, 76]",1200,Chicken,1,False,2,4920,6120,Chicken 1,2024-01-04 06:44:47.147383,2024-01-04 07:04:47.149239


In [ ]:
print("Max time: ",max(df['end_time']))
fig = px.timeline(df, x_start="start", x_end="end", y="task", color="name",hover_name='name', 
                    hover_data={'id':True,'duration':True,'start':False,'end':False,'task':False,'name':False,'source':False,'machine_id':False,'dependencies':True})
fig.update_yaxes(autorange="reversed")
fig.show()
#36300

Max time:  38100
